In [2]:
import json
import pandas as pd

league_id = '008da363-45af-4a07-87d2-f22567e27213'

from get_summoner_ids_from_league_id import get_summoner_ids_from_league_id
from get_puuid_by_summon_id import get_puuid_by_summon_id
from get_match_ids_by_puuid import get_match_ids_by_puuid
from get_match_timeline import get_match_timeline
from extract_events_from_timeline import extract_events_from_timeline

# Create rate limiter and get initial PUUids list

In [8]:
import time

class RateLimiter:
    def __init__(self, max_requests, period):
        self.max_requests = max_requests
        self.period = period
        self.timestamps = []

    def __call__(self):
        now = time.time()
        self.timestamps = [ts for ts in self.timestamps if ts > now - self.period]

        if len(self.timestamps) < self.max_requests:
            self.timestamps.append(now)
        else:
            oldest_request = min(self.timestamps)
            time_to_wait = self.period - (now - oldest_request)
            time.sleep(time_to_wait)
            self.timestamps.append(time.time())

rl_sec = RateLimiter(20, 1)  # 20 requests per 1 second
rl_min = RateLimiter(100, 120)  # 100 requests per 2 minutes

puuids = []
while len(puuids) < 20: 
    rl_sec()  # Apply rate limiting
    rl_min()  # Apply rate limiting

    summoner_ids = get_summoner_ids_from_league_id(league_id)
    
    for summoner_id in summoner_ids:
        rl_sec()  # Apply rate limiting
        rl_min()  # Apply rate limiting
        
        puuid = get_puuid_by_summon_id(summoner_id)
        puuids.append(puuid)

        if len(puuids) >= 20:
            break
    
print(f"Total puuids: {len(puuids)}")
print(puuids)


pd.DataFrame(puuids).to_csv('puuids.csv')

Total puuids: 20
['rOra4jm75Nj0y4-_7D7MVRfiBlpBXhpSonyI0rwwFacc0uV9fg2HC7PgC_nNJZ1WCXFL_EfTBrDLwQ', 'Cw5DTp0JHNYPKCvhFtw17wdGaFimvHUUckg4Fx8w2ZvYK7v9_mZtKHDOpInEGjGO5-5FZzFna-dDHg', 'ox60Cqe2-_wa_jLEVoy241kruFzWrD12QJd-RkxeBP5Mt3Kx5GA54dhPgfDTVtYnH7XbneQFgNi8aQ', 'RHZ59BUcE3vQxkwW_6x1sDkSXAjD9iR50Exf1nWDYYOzJdtpolEijub4KBwiqxamQYxUCfsRz2Ub9w', 'wwCb2LqGsnZUzOMCfC-YdN4tLb2SuPOwhw6yWPezsU9vG7FKoMGkazxBeWG_ebFYsynU6uQ70fqBuw', 'lnMmSlO27sHd6L0HgrYS5ivq8XMQDocSZytgMXsqMeu0dg3BiGxzfH7DGxAcBpuFpUDFIoDylKtvCg', 'rM1y8szWrtdn5qzlF-EA6S1jGpsZ65OC_4_6eaqmP26I5C8y2G-Z8wyqAc3G6RLCfJ9hTBBkSGMLFA', 'wylRJstyPPKMByoq_c4X59sItxo_Qhje0gXRZfQyl__YAa4O8OsjRhWpaNWTJ_byPdoLhalGzyyf3A', 'aFC0kPZY3mGhs0c-MT5blLeSuKiGzr9E9HJI9xEETp6piIgvNdYzx_xWo8f7emBqSmP_C6chMaVrvg', 'QH2QxpwEEEd0_BMQq1iydfZkVpVhDtCwe7QYeyNwd1vv5BWKJmlv2oYQuo_6phOOBKwB6U4ir6f2zA', '4aoPz1JnVaW0nY9N3rC-TwPR_5OORYX56Gpn2HDwe07bxrnT8yBIa1pof8FgL-eZ8aYGA9hYc8S8nQ', 'vRz9RPnMQ7uwkTVnNBZEF0F2ghwP9T_Xs_3hxKBdVrb4w0RSk3BTmbkGd4K866TivqC0G_9aVKJhwg'

# Get matchIds from PUUids

In [17]:
import random
import pandas as pd

# Create dict to hold match timelines
match_tl_dict = {}

# Create list to hold selected match IDs
selected_match_ids_list = []

# Iterate over PUUIDs
for puuid in puuids:
    # Apply rate limiting
    rl_sec()
    rl_min()

    # Get match IDs for each PUUID
    match_ids = get_match_ids_by_puuid(puuid)
    
    # Randomly select 10 (or up to 10) matches
    selected_match_ids = random.sample(match_ids, min(10, len(match_ids)))

    # Add selected match IDs to the list
    selected_match_ids_list.extend(selected_match_ids)

# Convert the list of selected match IDs to a DataFrame and save as CSV
pd.DataFrame(selected_match_ids_list, columns=['match_id']).to_csv('selected_match_ids.csv')
print('match_ids complete')

match_ids complete


# Get matchtimelines and save as JSON

In [19]:
import json
import os

# Create the 'match_timeline' folder if it doesn't exist
if not os.path.exists('match_timelines'):
    os.makedirs('match_timelines')

# Get match timelines
for match_id in selected_match_ids_list:
    # Apply rate limiting
    rl_sec()
    rl_min()

    # Get match timeline
    match_timeline = get_match_timeline(match_id)

    # Store the match timeline JSONs
    match_tl_dict[match_id] = match_timeline

    # Save the JSON data to a file inside the 'match_timeline' folder
    with open(os.path.join('match_timelines', f'{match_id}.json'), 'w') as json_file:
        json.dump(match_timeline, json_file)


# convert to tabular format

In [72]:
m_tl['info']['frames']

[{'events': [{'realTimestamp': 1609043040247,
    'timestamp': 0,
    'type': 'PAUSE_END'}],
  'participantFrames': {'1': {'championStats': {'abilityPower': 0,
     'armor': 27,
     'armorPen': 0,
     'armorPenPercent': 0,
     'attackDamage': 25,
     'attackSpeed': 100,
     'bonusArmorPenPercent': 0,
     'bonusMagicPenPercent': 0,
     'ccReduction': 0,
     'cooldownReduction': 0,
     'health': 602,
     'healthMax': 602,
     'healthRegen': 0,
     'lifesteal': 0,
     'magicPen': 0,
     'magicPenPercent': 0,
     'magicResist': 32,
     'movementSpeed': 340,
     'power': 0,
     'powerMax': 0,
     'powerRegen': 0,
     'spellVamp': 0},
    'currentGold': 500,
    'damageStats': {'magicDamageDone': 0,
     'magicDamageDoneToChampions': 0,
     'magicDamageTaken': 0,
     'physicalDamageDone': 0,
     'physicalDamageDoneToChampions': 0,
     'physicalDamageTaken': 0,
     'totalDamageDone': 0,
     'totalDamageDoneToChampions': 0,
     'totalDamageTaken': 0,
     'trueDamage

In [56]:
# Import necessary libraries
import os
import json
import pandas as pd

# Define function to read JSON file
def load_json(file_path):
    # Open and read the JSON file
    with open(file_path, 'r') as file:
        return json.load(file)

# Define function to process all JSON files in a folder
def extract_tables_from_folder(folder_path):
    # List to store all tables (processed data from each JSON file)
    tables = []

    # Loop over all files in the directory
    for file_name in os.listdir(folder_path):
        # Only process JSON files
        if file_name.endswith(".json"):
            # Construct the full file path
            file_path = os.path.join(folder_path, file_name)
            # Extract the match ID from the file name
            match_id = file_name.split('.')[0]

            # Load the JSON file
            m_tl = load_json(file_path)

            # Extract the events from the match timeline
            events_list = extract_events_from_timeline(m_tl)
            
            # Copy the events list and get the unique event types
            df = events_list.copy()
            event_types = df['type'].unique()

            # Compute the game's real start and end timestamps and the game length
            game_start_real_timestamp = int(events_list['realTimestamp'].iloc[0])
            game_end_real_timestamp = int(events_list['realTimestamp'].iloc[-1])
            game_length_ms = game_end_real_timestamp - game_start_real_timestamp

            # Remove the 'realTimestamp' column from the events list
            events_list.drop(columns='realTimestamp', inplace=True)

            # Prepare a dictionary to store event-specific dataframes
            events_dict = {}

            # Loop over all event types (except the first and last)
            for event_type in event_types[1:-1]:
                # Filter the dataframe for the current event type and drop all-NaN columns
                events_df = df[df['type'] == event_type].copy()
                events_df.dropna(axis=1, how='all', inplace=True)

                # Add a new 'match_id' column at the start of the dataframe
                events_df.insert(0, 'match_id', match_id)
                
                # Add the current event dataframe to the dictionary under the current event type
                events_dict[event_type] = events_df

            # Add the current match's data (events_dict) to the list of all tables
            tables.append(events_dict)

    tables.append(events_dict['game_metadata']='START' 'END' game_start_  game_end_)
    # Return the processed data from all matches
    return tables

# Usage example
folder_path = 'match_timelines'
extracted_tables = tabularize_json_from_folder(folder_path)

In [60]:
extracted_tables[0]['ITEM_PURCHASED']

,match_id,timestamp,type,itemId,participantId
1,NA1_4334085229,4522,ITEM_PURCHASED,3850.0,10.0
2,NA1_4334085229,4522,ITEM_PURCHASED,2003.0,10.0
3,NA1_4334085229,4522,ITEM_PURCHASED,2003.0,10.0
4,NA1_4334085229,5354,ITEM_PURCHASED,3340.0,10.0
6,NA1_4334085229,8520,ITEM_PURCHASED,2033.0,1.0
...,...,...,...,...,...
1250,NA1_4334085229,2041894,ITEM_PURCHASED,1026.0,5.0
1253,NA1_4334085229,2042259,ITEM_PURCHASED,3100.0,8.0
1270,NA1_4334085229,2068353,ITEM_PURCHASED,2140.0,9.0
1271,NA1_4334085229,2070030,ITEM_PURCHASED,3082.0,4.0


In [58]:
len(extracted_tables)

160

In [52]:
from extract_events_from_timeline import extract_events_from_timeline
import json

# Replace 'file_name.json' with the actual name of your JSON file
file_path = 'match_timelines/NA1_3720451304.json'
match_id = file_path.split('/')[-1].split('.')[0]

# Function to read the JSON file
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Load the JSON file and store it as m_tl
m_tl = load_json(file_path)

events_list = extract_events_from_timeline(m_tl)
df = events_list.copy()
event_types = df['type'].unique()

# Get start and end times
game_start_real_timestamp = int(events_list['realTimestamp'].iloc[0])
game_end_real_timestamp = int(events_list['realTimestamp'].iloc[-1])

game_length_ms = game_end_real_timestamp-game_start_real_timestamp
#print(f"Game Duration: 00:{int(game_length_ms/60000)}:{int((game_length_ms%60000)/1000)}:{int((game_length_ms%6000))}")

# redundant after storing timestamps above
events_list.drop(columns='realTimestamp',inplace=True)

events_dict = {}
# exclude PAUSE_END and GAME_END
for event_type in event_types[1:-1]:
    events_df = df[df['type'] == event_type].copy()
    events_df.dropna(axis=1, how='all', inplace=True)
    events_df.insert(0, 'match_id', match_id) 
    events_dict[event_type] = events_df

NA1_3720451304
Game Duration: 00:20:12:895


# connect to postgresql

In [55]:
from sqlalchemy import text
from sqlalchemy.exc import OperationalError
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

def read_txt(filename):
    with open(filename, 'r') as file:
        return file.read().strip()

password = read_txt('postgres_pw.txt')
host = 'localhost'
database_name = 'loldb'
port = '5432'
engine = create_engine(f'postgresql://postgres:{password}@{host}:{port}/{database_name}')

# Try connecting and executing a simple query
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Successfully connected!")
except OperationalError as e:
    print(f"Connection failed: {e}")

from sqlalchemy import create_engine, MetaData

# Get the metadata of the database
metadata = MetaData()
metadata.reflect(bind=engine)

# List all table names
table_names = metadata.tables.keys()

print("Tables in the database:")
for table_name in table_names:
    print(table_name)

Successfully connected!
Tables in the database:
MATCH_TIMELINE


# Split JSONs into tables and load to postgreSQL

In [7]:
from sqlalchemy import text
from sqlalchemy.exc import OperationalError

try:
    with engine.connect() as connection:
        result = connection.execute(text('SELECT * FROM "MATCH_TIMELINE"'))
        print("Successfully connected!")

        # Fetch all records and print them
        rows = result.fetchall()

        if not rows:
            print("The table is empty. No records to fetch.")
        else:
            for row in rows:
                print(row)

except OperationalError as e:
    print(f"Connection failed: {e.orig}")

Successfully connected!
The table is empty. No records to fetch.


In [9]:
import os
import json
from sqlalchemy import create_engine, Table, Column, Integer, String, JSON, MetaData
from sqlalchemy.sql import insert

# Define the table structure
metadata = MetaData()
match_timeline_table = Table(
    'MATCH_TIMELINE',
    metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('match_id', String, nullable=False),
    Column('timeline_json', JSON, nullable=False)
)

# Create the table in the database (if it doesn't exist)
metadata.create_all(engine)

total_rows_added = 0

# Connect to the database
with engine.connect() as connection:

    # Loop through each file in the match_timelines directory
    for filename in os.listdir('match_timelines'):

        # Ensure the file is a JSON file
        if filename.endswith('.json'):
            # Open the file and load the JSON data
            with open(os.path.join('match_timelines', filename), 'r') as json_file:
                match_timeline = json.load(json_file)

            # Create an INSERT statement for the match_timeline
            stmt = insert(match_timeline_table).values(match_id=match_id, timeline_json=match_timeline)

            # Execute the statement and increment the total_rows_added by the number of rows affected
            try:
                result = connection.execute(stmt)
                rows_added = result.rowcount
                total_rows_added += rows_added
                print(f"Rows added: {rows_added}")
            except Exception as e:
                print(f"Error occurred when executing INSERT statement: {e}")

print(f"Total rows added: {total_rows_added}")

Error occurred when executing INSERT statement: (psycopg2.errors.NumericValueOutOfRange) integer out of range

[SQL: INSERT INTO "MATCH_TIMELINE" (match_id, timeline_json) VALUES (%(match_id)s, %(timeline_json)s) RETURNING "MATCH_TIMELINE".id]
[parameters: {'match_id': 4518725197, 'timeline_json': '{"metadata": {"dataVersion": "2", "matchId": "NA1_4334085229", "participants": ["pEzq-TwsSVlh_qVqqKbSP-LcHaCSlPSHzYoksh86q-DzJ3F1-uOXY-VL_v4oKc7WsXzN ... (767365 characters truncated) ... iGxzfH7DGxAcBpuFpUDFIoDylKtvCg"}, {"participantId": 10, "puuid": "lbpYhDIEMOhsFmgtcg8JgiJ3ONencq5fiKJqxWxwE4HCpXK3jnUZV9CInwqsNzwFLE9ztahbpg4zhw"}]}}'}]
(Background on this error at: https://sqlalche.me/e/20/9h9h)
Error occurred when executing INSERT statement: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: INSERT INTO "MATCH_TIMELINE" (match_id, timeline_json) VALUES (%(match_id)s, %(timeline_json)s) RETURNING "MATCH_TIMELINE"

In [ ]:
# import json
# league_id = '008da363-45af-4a07-87d2-f22567e27213'

# summoner_ids = get_summoner_ids_from_league_id(league_id)

# puuid = get_puuid_by_summon_id(summoner_ids[0])

# match_ids = get_match_ids_by_puuid(puuid)

# match_timeline = get_match_timeline(match_ids[0])

# timeline_events = extract_events_from_timeline(match_timeline)